In [2]:
!pip install -U -q keras-nlp
!pip install --upgrade keras
!pip install kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.4/548.4 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.1 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [1]:
import os
from google.colab import userdata
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [3]:
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [4]:
import os
os.environ["KERAS_BACKEND"] = "jax"

In [5]:
%%capture
import kagglehub

path = kagglehub.model_download("mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf")

In [6]:
print("Path to model files:", path)

Path to model files: /root/.cache/kagglehub/models/mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf/1


In [7]:
!pip install -q langchain langchain_community sentence-transformers transformers accelerate einops xformers bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5

In [9]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00


In [10]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.document_loaders import DataFrameLoader
import pandas as pd
import torch
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from langchain.llms import HuggingFacePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from datasets import Dataset
import json

In [11]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)

In [12]:
%%time
model = AutoModelForCausalLM.from_pretrained(
    "/root/.cache/kagglehub/models/mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf/1",
    quantization_config = bnb_config,
    do_sample=True,
)
tokenizer = AutoTokenizer.from_pretrained("/root/.cache/kagglehub/models/mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf/1")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

query_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    temperature=0.5,
    task="text-generation",
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=4000,
    batch_size=4,
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 24.3 s, sys: 9.67 s, total: 33.9 s
Wall time: 14.6 s


In [13]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

<ipython-input-13-a64ef6314ad8>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=query_pipeline)


In [14]:
!pip install opendatasets

In [15]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/jscotthorn/kyag-opinions-and-decisions")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: syedosamaalishah092
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/jscotthorn/kyag-opinions-and-decisions


100%|██████████| 238M/238M [00:14<00:00, 17.4MB/s]


In [16]:
import pandas as pd

df = pd.read_csv("/content/kyag-opinions-and-decisions/kyag_opinions.csv")

In [17]:
filtered_df = df.loc[df['opinion_type'] != 'Opinion']

In [18]:
filtered_df.describe()

,year,lexis_number
count,7787.000000,7771.000000
mean,2005.199435,152.692446
std,10.890998,105.134410
min,1977.000000,1.000000
25%,1998.000000,70.000000
50%,2006.000000,141.000000
75%,2014.000000,213.000000
max,2022.000000,772.000000


In [19]:
filtered_df.head()

,date,date_string,year,lexis_number,lexis_title_extra,citation_oag,opinion_type,decision_text,full_text,full_html,references,reference_map,case_citations,decision_html_array,citation_lexis_array
0,2000-02-21,2000-02-21,2000,5.0,NaN,00-OMD-063,Open Meetings Decision,Request By:\n \n\n [NO REQUESTBY IN ORIGINAL]\...,KYOAG\n \n\n 2000 Ky. AG LEXIS 5::February 21...,"<head>\n <meta content=""text/html; charset=utf...","[""94-OMD-106"", ""98-OMD-044"", ""94-OMD-050"", ""OA...","{""94-OMD-106"": [""94-OMD-106""], ""98-OMD-044"": [...","[""955 S.W.2d 921"", ""790 S.W.2d 450""]","[""<p class=\""p12\"">\n <b>\n Request By:\n </b...","[""2000 Ky. AG LEXIS 5""]"
1,2000-02-21,2000-02-21,2000,6.0,NaN,00-OMD-064,Open Meetings Decision,Request By:\n \n\n [NO REQUESTBY IN ORIGINAL]\...,KYOAG\n \n\n 2000 Ky. AG LEXIS 6::February 21...,"<head>\n <meta content=""text/html; charset=utf...","[""93-OMD-056"", ""94-OMD-022"", ""98-OMD-147"", ""95...","{""93-OMD-056"": [""93-OMD-56""], ""94-OMD-022"": [""...","[""955 S.W.2d 921""]","[""<p class=\""p8\"">\n <b>\n Request By:\n </b>...","[""2000 Ky. AG LEXIS 6""]"
2,2000-02-25,2000-02-25,2000,7.0,NaN,00-OMD-065,Open Meetings Decision,Request By:\n \n\n [NO REQUESTBY IN ORIGINAL]\...,KYOAG\n \n\n 2000 Ky. AG LEXIS 7::February 25...,"<head>\n <meta content=""text/html; charset=utf...","[""92-OMD-184""]","{""92-OMD-184"": [""92-OMD-184""]}","[""955 S.W.2d 921"", ""790 S.W.2d 450""]","[""<p class=\""p8\"">\n <b>\n Request By:\n </b>...","[""2000 Ky. AG LEXIS 7""]"
3,2000-03-16,2000-03-16,2000,73.0,NaN,00-OMD-080,Open Meetings Decision,Opinion\n \n\n Opinion By:\n \n\n Albert B. C...,"KYOAG\n \n\n 2000 Ky. AG LEXIS 73::March 16, ...","<head>\n <meta content=""text/html; charset=utf...","[""98-OMD-075"", ""98-OMD-074"", ""96-OMD-216"", ""94...","{""98-OMD-075"": [""98-OMD-75""], ""98-OMD-074"": [""...","[""955 S.W.2d 921""]","[""<p class=\""p1\"">\n <b>\n Opinion\n </b>\n</...","[""2000 Ky. AG LEXIS 73""]"
4,2000-03-21,2000-03-21,2000,106.0,NaN,00-OMD-086,Open Meetings Decision,Opinion\n \n\n Opinion By:\n \n\n Albert B. C...,"KYOAG\n \n\n 2000 Ky. AG LEXIS 106::March 21,...","<head>\n <meta content=""text/html; charset=utf...","[""93-OMD-049"", ""97-OMD-110"", ""94-OMD-103"", ""97...","{""93-OMD-049"": [""93-OMD-49""], ""97-OMD-110"": [""...",[],"[""<p class=\""p1\"">\n <b>\n Opinion\n </b>\n</...","[""2000 Ky. AG LEXIS 106""]"


In [21]:
filtered_df.to_csv('/content/kyag-opinions-and-decisions/kyag_sunshine_decisions.csv', index=False)

In [23]:
dtype_dict = {
    'citation_lexis': 'object',
    'citation_oag': 'object',
    'date_string': 'object',
    'year': 'object',
    'lexis_number': 'object',
    'opinion_type': 'object',
    'decision_text': 'object',
    'decision_html': 'object',
    'full_text': 'object',
    'full_html': 'object',
    'references': 'object',
}

df_decisions = pd.read_csv('/content/kyag-opinions-and-decisions/kyag_sunshine_decisions.csv', dtype=dtype_dict)

df_decisions = df_decisions.loc[df_decisions['year'] == "2022"]

print(df_decisions.head())

            date date_string  year lexis_number  lexis_title_extra  \
5319  2022-01-27  2022-01-27  2022         17.0                NaN   
5320  2022-04-07  2022-04-07  2022         65.0                NaN   
5321  2022-08-16  2022-08-16  2022        168.0                NaN   
5322  2022-08-24  2022-08-24  2022        180.0                NaN   
5323  2022-08-31  2022-08-31  2022        181.0                NaN   

     citation_oag            opinion_type  \
5319   22-OMD-017  Open Meetings Decision   
5320   22-OMD-057  Open Meetings Decision   
5321   22-OMD-163  Open Meetings Decision   
5322   22-OMD-171  Open Meetings Decision   
5323   22-OMD-177  Open Meetings Decision   

                                          decision_text  \
5319  Opinion\n \n\n  Opinion By:\n \n\n Daniel Came...   
5320  Request By:\n \n\n Brennan D. Crain\n \n\n Dan...   
5321  Opinion\n \n\n  Opinion By:\n \n\n Daniel Came...   
5322  Opinion\n \n\n  Opinion By:\n \n\n Daniel Came...   
5323  Opinion

In [24]:
df_decisions.describe()

,lexis_title_extra
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [25]:
segment_prompt = """
You are an AI attorney shepardizing opinions of the attorney general of Kentucky.
You will be provided text of a decision with a dictionary of opinions mapped to a list of strings used to refer to them in the text.
You must ensure we accurately note in JSON whether previous opinions were (F) Followed; (M) Modified; (W) Withdrawn; or (R) Reversed.
Return only a JSON object keyed by opinion citation with a value of F, M, W, R, or "N" for Not Found. Your response must be correct JSON.


example input:
{
    "decision": "We hereby affirm our decision in 98-ORD-042 and modify our previous decision 20-ORD-014 to the extent...",
    "references": {"20-ORD-014":["20-ORD-014"],"98-ORD-042":["98-ORD-042"],"OAG 78-01":["78-01"]}
}
example output:
{"20-ORD-014": "F","98-ORD-042": "M","OAG 78-01": "N"}


input:
{
    "decision": "{query}",
    "references": {references}
}
output:
"""

def shepardize_opinions(row):
    try:
        text = row['decision_text']
        filled_prompt = segment_prompt.replace("{decision}", json.dumps(text)).replace("{references}", row['references'])
        result = llm(prompt=filled_prompt)
        return result.replace(filled_prompt, '').split('}')[0] + '}'
    except Exception as e:
        print(f'Error while processing row\n{text}\nException: {e}')
    return ['', '']

In [26]:
print(df_decisions.iloc[0]['references'])
print(shepardize_opinions(df_decisions.iloc[0]))

[]


<ipython-input-25-cb57c8203eb2>:29: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm(prompt=filled_prompt)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


{"query": "Not Found"}


In [31]:
df_decisions['treatments'] = df_decisions.apply(shepardize_opinions, axis=1)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [28]:
df_decisions.head()

,date,date_string,year,lexis_number,lexis_title_extra,citation_oag,opinion_type,decision_text,full_text,full_html,references,reference_map,case_citations,decision_html_array,citation_lexis_array,treatments
5319,2022-01-27,2022-01-27,2022,17.0,NaN,22-OMD-017,Open Meetings Decision,Opinion\n \n\n Opinion By:\n \n\n Daniel Came...,KYOAG\n \n\n 2022 KY. AG LEXIS 17::January 27...,"<head>\n <meta content=""text/html; charset=utf...",[],{},[],"[""<p class=\""p1\"">\n <b>\n Opinion\n </b>\n</...","[""2022 KY. AG LEXIS 17""]","{""query"": ""Not Found""}"
5320,2022-04-07,2022-04-07,2022,65.0,NaN,22-OMD-057,Open Meetings Decision,Request By:\n \n\n Brennan D. Crain\n \n\n Dan...,"KYOAG\n \n\n 2022 KY. AG LEXIS 65::April 7, 2...","<head>\n <meta content=""text/html; charset=utf...",[],{},"[""373 S.W.3d 438""]","[""<p class=\""p9\"">\n <b>\n Request By:\n </b>...","[""2022 KY. AG LEXIS 65""]","{""query"": ""Not Found""}"
5321,2022-08-16,2022-08-16,2022,168.0,NaN,22-OMD-163,Open Meetings Decision,Opinion\n \n\n Opinion By:\n \n\n Daniel Came...,KYOAG\n \n\n 2022 KY. AG LEXIS 168::August 16...,"<head>\n <meta content=""text/html; charset=utf...","[""21-OMD-086""]","{""21-OMD-086"": [""21-OMD-086""]}",[],"[""<p class=\""p1\"">\n <b>\n Opinion\n </b>\n</...","[""2022 KY. AG LEXIS 168""]","{""21-OMD-086"": ""N""}"
5322,2022-08-24,2022-08-24,2022,180.0,NaN,22-OMD-171,Open Meetings Decision,Opinion\n \n\n Opinion By:\n \n\n Daniel Came...,KYOAG\n \n\n 2022 KY. AG LEXIS 180::August 24...,"<head>\n <meta content=""text/html; charset=utf...","[""22-OMD-017""]","{""22-OMD-017"": [""22-OMD-17""]}",[],"[""<p class=\""p1\"">\n <b>\n Opinion\n </b>\n</...","[""2022 KY. AG LEXIS 180""]","{""22-OMD-017"": ""N""}"
5323,2022-08-31,2022-08-31,2022,181.0,NaN,22-OMD-177,Open Meetings Decision,Opinion\n \n\n Opinion By:\n \n\n Daniel Came...,KYOAG\n \n\n 2022 KY. AG LEXIS 181::August 31...,"<head>\n <meta content=""text/html; charset=utf...","[""22-ORD-165"", ""22-ORD-078"", ""18-OMD-163"", ""12...","{""22-ORD-165"": [""22-ORD-165""], ""22-ORD-078"": [...","[""607 S.W.3d 586"", ""499 S.W.3d 286""]","[""<p class=\""p1\"">\n <b>\n Opinion\n </b>\n</...","[""2022 KY. AG LEXIS 181""]","{""22-ORD-165"": ""F"",""22-ORD-078"": ""F"",""18-OMD-1..."


In [29]:
df_decisions.describe()

,lexis_title_extra
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [30]:
df_decisions.to_csv('/content/working/sunshine-ag-annotations.csv', index=False)